In [1]:
!pip install -q llama-index
!pip install -q openai
!pip install -q transformers
!pip install -q accelerate
!pip install -q optimum[exporters]
!pip install -q InstructorEmbedding
!pip install -q sentence_transformers
!pip install -q pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 850.8/850.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.3/484.3 kB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import openai
import os
openai.api_key = "Enter Your OpenAI Key"

In [4]:
from llama_index.llms import OpenAI
from llama_index import VectorStoreIndex, SimpleDirectoryReader
from IPython.display import Markdown, display

In [5]:
documents = SimpleDirectoryReader("data").load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()
response = query_engine.query("What did the author do growing up? ")

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
from llama_index import ServiceContext, set_global_service_context

llm = OpenAI(
    model = "gpt-3.5-turbo",
    temperature = 0,
    max_tokens = 256
)
service_context = ServiceContext.from_defaults(
    llm = llm,
    chunk_size = 800,
    chunk_overlap = 20
)
index = VectorStoreIndex.from_documents(
    documents,
    service_context = service_context
)

## Customize Embeddings model

In [7]:
from llama_index import ServiceContext
from llama_index.embeddings import OpenAIEmbedding

In [8]:
openai_embed = OpenAIEmbedding()
service_context = ServiceContext.from_defaults(embed_model=openai_embed)

In [9]:
text_embeddings = openai_embed.get_text_embedding("AI is awesome!")

In [10]:
len(text_embeddings)

1536

### Local embeddings from hugging face
Massive Text Embedding Benchmark (MTEB) [Leaderboard](https://huggingface.co/spaces/mteb/leaderboard)

### bge embedding

In [11]:
from llama_index.embeddings import HuggingFaceEmbedding

In [12]:
embed_model_bge = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
text_embeddings = embed_model_bge.get_text_embedding("AI is awesome!")
print(len(text_embeddings))

384


### Instructor embedding

In [13]:
from llama_index.embeddings import InstructorEmbedding

In [14]:
embed_model_inst = InstructorEmbedding(model_name="hkunlp/instructor-base")
text_embeddings = embed_model_inst.get_text_embedding("AI is awesome!")
print(len(text_embeddings))

load INSTRUCTOR_Transformer
max_seq_length  512
768


## Speed Benchmarking

In [15]:
!curl https://www.ipcc.ch/report/ar6/wg2/downloads/report/IPCC_AR6_WGII_Chapter03.pdf --output IPCC_AR6_WGII_Chapter03.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 20.7M  100 20.7M    0     0   249k      0  0:01:25  0:01:25 --:--:--  257k


In [16]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader

In [17]:
documents = SimpleDirectoryReader(
    input_files = ["IPCC_AR6_WGII_Chapter03.pdf"]
).load_data()

In [18]:
print(f"Number of documents: {len(documents)}")

Number of documents: 172


### OpenAI embeddings

In [19]:
embed_model_openai = OpenAIEmbedding()
service_context = ServiceContext.from_defaults(embed_model=embed_model_openai)

In [20]:
%%timeit -r 2 -n 2
index = VectorStoreIndex.from_documents(
    documents, service_context=service_context, show_progress=True
)

Parsing documents into nodes:   0%|          | 0/172 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/428 [00:00<?, ?it/s]

Parsing documents into nodes:   0%|          | 0/172 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/428 [00:00<?, ?it/s]

Parsing documents into nodes:   0%|          | 0/172 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/428 [00:00<?, ?it/s]

Parsing documents into nodes:   0%|          | 0/172 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/428 [00:00<?, ?it/s]

18.2 s ± 697 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)


### BGE embeddings

In [21]:
service_context = ServiceContext.from_defaults(embed_model=embed_model_bge)

In [22]:
%%timeit -r 2 -n 2
index = VectorStoreIndex.from_documents(
    documents, service_context=service_context, show_progress=True)

Parsing documents into nodes:   0%|          | 0/172 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/428 [00:00<?, ?it/s]

Parsing documents into nodes:   0%|          | 0/172 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/428 [00:00<?, ?it/s]

Parsing documents into nodes:   0%|          | 0/172 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/428 [00:00<?, ?it/s]

Parsing documents into nodes:   0%|          | 0/172 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/428 [00:00<?, ?it/s]

8.61 s ± 40.2 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)


### Instructor embeddings

In [23]:
service_context = ServiceContext.from_defaults(embed_model=embed_model_inst)

In [24]:
%%timeit -r 2 -n 2
index = VectorStoreIndex.from_documents(
    documents, service_context=service_context, show_progress=True)

Parsing documents into nodes:   0%|          | 0/172 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/428 [00:00<?, ?it/s]

Parsing documents into nodes:   0%|          | 0/172 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/428 [00:00<?, ?it/s]

Parsing documents into nodes:   0%|          | 0/172 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/428 [00:00<?, ?it/s]

Parsing documents into nodes:   0%|          | 0/172 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/428 [00:00<?, ?it/s]

18.8 s ± 648 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)
